<a href="https://colab.research.google.com/github/sumandutta8877/DATA-Analytics/blob/suman/DA_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data from GitHub

In [1]:
!git clone 'https://github.com/sumandutta8877/Data_DA.git'    # Run this Line for only one time

Cloning into 'Data_DA'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


In [37]:
import pandas as pd
df_train = pd.read_csv('/content/Data_DA/Data/Train_data.csv')
df_test = pd.read_csv('/content/Data_DA/Data/Test_data.csv')
df_sample = pd.read_csv('/content/Data_DA/Data/Sample_submission.csv')
#print(df_train.head)
#print(df_test.head)
#print(df_sample.head)
df1 = df_train            # df1 is the training dataframe
df2 = df_test             # df2 is the test dataframe

# Start Processing

In [38]:
df1['mode'] = df1['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df1['explicit'] = df1['explicit'].apply(lambda x: 0.2 if x == 'No' else 1.2)
df2['mode'] = df2['mode'].apply(lambda x: 2 if x == 'Major' else 1)
df2['explicit'] = df2['explicit'].apply(lambda x: 0.2 if x == 'No' else 1.2)
df1.loc[(df1['popularity'] == 'very low'),'ratings'] = 1
df1.loc[(df1['popularity'] == 'low'),'ratings'] = 2
df1.loc[(df1['popularity'] == 'average'),'ratings'] = 3
df1.loc[(df1['popularity'] == 'high'),'ratings'] = 4
df1.loc[(df1['popularity'] == 'very high'),'ratings'] = 5
del df1['popularity']
del df1['year']
del df2['year']
df = df1['id']
df['temp1'] = df1['release_date'].str[0:2].astype(int)
df['temp2'] = df1['release_date'].str[3:5].astype(int)
df['temp3'] = df1['release_date'].str[6:10].astype(int)
df1['release_date'] = 737300 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today
df = df2['id']
df['temp1'] = df2['release_date'].str[0:2].astype(int)
df['temp2'] = df2['release_date'].str[3:5].astype(int)
df['temp3'] = df2['release_date'].str[6:10].astype(int)
df2['release_date'] = 737300 - (df['temp1']+30.45*df['temp2']+365*df['temp3'])  # number of days old from today
print(df1)

          id  acousticness  danceability  ...  valence  duration-min  ratings
0       2015        0.9490        0.2350  ...  0.03980           3.0      1.0
1      15901        0.8550        0.4560  ...  0.85900           2.4      2.0
2       9002        0.8270        0.4950  ...  0.70900           2.6      1.0
3       6734        0.6540        0.6430  ...  0.96400           2.4      2.0
4      15563        0.7380        0.7050  ...  0.78500           3.4      3.0
...      ...           ...           ...  ...      ...           ...      ...
12222  15343        0.0408        0.8090  ...  0.74400           3.4      3.0
12223   1701        0.9120        0.4510  ...  0.70100           2.0      5.0
12224   3351        0.3280        0.5510  ...  0.89000           2.5      2.0
12225   8879        0.1220        0.0608  ...  0.00558           3.1      4.0
12226   9711        0.0380        0.3890  ...  0.33400           3.1      4.0

[12227 rows x 16 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(df2)

         id  acousticness  danceability  ...  valence  year  duration-min
0      4943       0.03360         0.523  ...    0.346  2014           3.8
1     16165       0.76500         0.687  ...    0.830  1972           3.8
2      6774       0.36300         0.536  ...    0.289  1998           4.4
3      9959       0.00463         0.890  ...    0.970  1980           3.9
4     13818       0.00187         0.555  ...    0.882  2007           3.2
...     ...           ...           ...  ...      ...   ...           ...
3995   6367       0.00019         0.272  ...    0.564  1981           3.5
3996  12061       0.10600         0.444  ...    0.747  2017           3.1
3997  15802       0.64900         0.556  ...    0.604  1994           3.3
3998  11746       0.99500         0.378  ...    0.158  1946           2.6
3999   8441       0.02620         0.534  ...    0.145  2017           3.9

[4000 rows x 16 columns]
